In [10]:
import json
from tqdm import tqdm
with open('/home/nkama/LLM_and_RAG_Course/LLM_and_RAG-/personal_assg_project/interview_qa.json', 'rt') as f_in:
    documents = json.load(f_in)

In [17]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-MiniLM-L6-cos-v1'

model = SentenceTransformer(model_name)

/home/nkama/.pyenv/versions/3.10.6/envs/llmMath/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nkama/.pyenv/versions/3.10.6/envs/llmMath/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
encoded_question = model.encode('I just discovered the course. Can I still join?')

In [23]:
len(v)

384

In [24]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [25]:
from personal_assg_project.modular_rag import encode_and_index_documents, create_vector_index, vector_elastic_search

In [1]:
create_vector_index(es_client)


NameError: name 'create_vector_index' is not defined

In [27]:
encode_and_index_documents(es_client, model, documents)


Indexing documents: 100%|██████████| 948/948 [00:07<00:00, 124.59it/s]


In [28]:
vector_elastic_search(es_client, "question_text_vector", encoded_question, "data-engineering-zoomcamp")

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': 'a482086d'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

In [3]:
query = 'I just discovered the course. Can I still join it?'

In [8]:
v_q = model.encode(query)

In [47]:
# Function to retrieve responses for test data using vector search
def question_text_vector(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return vector_elastic_search(es_client, "question_text_vector", v_q, course)

In [3]:
import pandas as pd

In [32]:
# Load test data
df_test_data = pd.read_csv('/home/nkama/LLM_and_RAG_Course/LLM_and_RAG-/data/ground-truth-data.csv')
df_test_data.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [33]:
#turn dataframe to dict.
test_data = df_test_data.to_dict(orient='records')

In [14]:
test_data[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

**We use two metrices to evaluate the retrieval performance**

- Hit Rate
- MRR

In [5]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [7]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [8]:
#Evaluate retrieval
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [48]:
evaluate(test_data, test_data_vector_knn)

100%|██████████| 4627/4627 [01:34<00:00, 49.05it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

ES text only: 0.7395720769397017, 0.6032418413658963

In [41]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return vector_elastic_search(es_client, "text_vector", v_q, course)

In [42]:
evaluate(test_data, test_data_vector_knn)

100%|██████████| 4627/4627 [01:36<00:00, 48.17it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

In [46]:
def question_vector(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return vector_elastic_search(es_client, "question_vector", v_q, course)

evaluate(test_data, test_data_vector_knn)

100%|██████████| 4627/4627 [01:33<00:00, 49.24it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}